<a href="https://colab.research.google.com/github/lingyu001/Recommendation/blob/master/YoutubeDNN_breakdown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
! wget http://files.grouplens.org/datasets/movielens/ml-1m.zip -O ./ml-1m.zip
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py -O preprocess.py
! wget https://raw.githubusercontent.com/lingyu001/Recommendation/master/model_utils.py -O model_utils.py
! unzip -o ml-1m.zip 

--2022-10-05 19:51:51--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘./ml-1m.zip’

./ml-1m.zip         100%[===================>]   5.64M  18.2MB/s    in 0.3s    

2022-10-05 19:51:51 (18.2 MB/s) - ‘./ml-1m.zip’ saved [5917549/5917549]

--2022-10-05 19:51:51--  https://raw.githubusercontent.com/lingyu001/Recommendation/master/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6687 (6.5K) [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]   6.53K

In [317]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 5.0 MB/s 
time: 1.61 ms (started: 2022-10-05 21:42:44 +00:00)


In [49]:
! pip install -q deepmatch

In [50]:
import pandas as pd
from preprocess import gen_data_set, gen_model_input
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from model_utils import DNN, EmbeddingIndex, NoMask, PoolingLayer, l2_normalize,\
 reduce_mean, SampledSoftmaxLayer, get_item_embedding, sampledsoftmaxloss

In [51]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data

In [253]:
data_path = "./"

unames = ['user_id','gender','age','occupation','zip']
user = pd.read_csv(data_path+'ml-1m/users.dat',sep='::',header=None,names=unames)
rnames = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv(data_path+'ml-1m/ratings.dat',sep='::',header=None,names=rnames)
mnames = ['movie_id','title','genres']
movies = pd.read_csv(data_path+'ml-1m/movies.dat',sep='::',header=None,names=mnames,encoding="unicode_escape")
movies['genres'] = list(map(lambda x: x.split('|')[0], movies['genres'].values))

data = pd.merge(pd.merge(ratings,movies),user)#.iloc[:10000] # taking 1w
# data = data[data['user_id'].isin([4169,1680,4277])]
print(data.shape)
print(data.user_id.nunique())
print(data.movie_id.nunique())
data.head(2)

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


(1000209, 10)
6040
3706


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation,F,1,10,48067


# Feature Engineering

In [254]:
sparse_features = ["movie_id", "user_id",
                    "gender", "age", "occupation", "zip", "genres"]

SEQ_LEN = 50
NUMBER_NEG_SAMPLE = 1 # used for generate 1:1 positive:negative samples


In [255]:
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    data[feature] = lbe.fit_transform(data[feature]) + 1 # add one to all the encoded categories labels
    feature_max_idx[feature] = data[feature].max() + 1

In [256]:
feature_max_idx

{'movie_id': 3707,
 'user_id': 6041,
 'gender': 3,
 'age': 8,
 'occupation': 22,
 'zip': 3440,
 'genres': 19}

In [257]:
data.head(2)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
0,1,1105,5,978300760,One Flew Over the Cuckoo's Nest (1975),8,1,1,11,1589
1,1,640,3,978302109,James and the Giant Peach (1996),3,1,1,11,1589


In [258]:
user_profile = data[["user_id", "gender", "age", "occupation", "zip"]].drop_duplicates('user_id')
user_profile.set_index("user_id", inplace=True)
item_profile = data[["movie_id"]].drop_duplicates('movie_id')
user_profile.shape, item_profile.shape

((6040, 4), (3706, 1))

In [259]:
user_item_list = data.groupby("user_id")['movie_id'].apply(list)
user_item_list.head()

user_id
1    [1105, 640, 854, 3178, 2163, 1108, 1196, 2600,...
2    [1105, 2890, 2129, 1783, 1118, 1849, 1155, 126...
3    [2163, 1108, 1179, 1782, 254, 2899, 628, 1121,...
4    [1026, 2489, 254, 1849, 3236, 1121, 467, 1107,...
5    [3178, 2163, 859, 2890, 1575, 2558, 145, 2489,...
Name: movie_id, dtype: object

In [260]:
train_set, test_set = gen_data_set(data, SEQ_LEN, NUMBER_NEG_SAMPLE)

100%|██████████| 6040/6040 [00:26<00:00, 227.85it/s]


8 8


In [261]:
# reviewerID,  - user id
# pos_list[i], - movie id
# 1, - label y
# hist[::-1][:seq_len], - most recent watched movie id list before watching this movie, up to 50
# seq_len
# genres_hist[::-1][:seq_len], - most recent watched movie's genres list before watching this movie, up to 50
# genres_list[i], - this movie's genre
# rating_list[i] - this movie's rating
# train_set[12]

In [262]:
train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)
train_label.shape, train_model_input['user_id'].shape, test_model_input['user_id'].shape, test_label.shape

/content/preprocess.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_seq_genres = np.array([line[5] for line in train_set])


((1976258,), (1976258,), (6040,), (6040,))

# Create Embeddings

In [263]:
from collections import Counter
train_counter = Counter(train_model_input['movie_id'])
item_count = [train_counter.get(i,0) for i in range(feature_max_idx['movie_id'])]
len(train_counter), type(train_counter), len(item_count)

(3706, collections.Counter, 3707)

In [264]:
import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros
from tensorflow.python.keras.layers import Embedding, Input, Lambda
from tensorflow.python.keras.regularizers import l2
if tf.__version__ >= '2.0.0':
    tf.compat.v1.disable_eager_execution()
else:
    K.set_learning_phase(True)

### Create Feature Embeddings



In [265]:
# item features
item_feature_columns = ['movie_id']
# User features
user_feature_columns = ['user_id',
                        "gender",
                        "age",
                        "occupation",
                        "zip",
                        'hist_movie_id',
                        'hist_genres',
                        'hist_len'
                        ]


In [266]:
feature_max_idx

{'movie_id': 3707,
 'user_id': 6041,
 'gender': 3,
 'age': 8,
 'occupation': 22,
 'zip': 3440,
 'genres': 19}

In [267]:
user_sparse_feature_columns = user_feature_columns[0:5]
user_sparse_feature_columns

['user_id', 'gender', 'age', 'occupation', 'zip']

In [268]:
user_seq_sparse_feature_columns = user_feature_columns[5:-1]
user_seq_sparse_feature_columns 

['hist_movie_id', 'hist_genres']

In [269]:
# embedding input and output specification
embeddings_initializer = RandomNormal(mean=0.0, stddev=0.0001, seed=2020)
embedding_dim = 32
l2_reg = 1e-6
# use a dict to collect embeddings
sparse_embedding = {}

In [270]:
# user embeddings
for name in user_sparse_feature_columns: 
  emb = Embedding(
      input_dim = feature_max_idx[name], 
      output_dim = 16,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + name
  )
  emb.trainable = True
  sparse_embedding[name] = emb
for name in user_seq_sparse_feature_columns: 
   emb = Embedding(feature_max_idx[name[5:]], embedding_dim,
                            embeddings_initializer=embeddings_initializer,
                            embeddings_regularizer=l2(
                                l2_reg),
                            name='sparse_' + 'seq_emb_'  + name,
                            mask_zero=False) # True
   emb.trainable = True
   sparse_embedding[name] = emb

In [271]:
# item embeddings
item_feature_name = item_feature_columns[0] # only include item id 
emb = Embedding(
      input_dim = feature_max_idx[item_feature_name], 
      output_dim = embedding_dim,
      embeddings_initializer=embeddings_initializer,
      embeddings_regularizer=l2(l2_reg),
      name='sparse_' + 'emb_' + item_feature_name
  )
sparse_embedding[item_feature_name] = emb

In [272]:
sparse_embedding # include both user and item embedding layers

{'user_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ff57d024910>,
 'gender': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ff578f2e710>,
 'age': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ff578f2c3d0>,
 'occupation': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ff578f2c390>,
 'zip': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ff4fc8ade50>,
 'hist_movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ff578f2ead0>,
 'hist_genres': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ff505ef80d0>,
 'movie_id': <tensorflow.python.keras.layers.embeddings.Embedding at 0x7ff578f2fc90>}

# Create User Inputs


## Create Inputs Tensor

In [273]:
# user features inputs
from collections import OrderedDict
user_features = OrderedDict()
for name in user_sparse_feature_columns:
  user_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
for name in user_seq_sparse_feature_columns:
  user_features[name] = Input(
                shape=(SEQ_LEN,), 
                name=name, 
                dtype="int32"
                )
length_name = 'hist_len'
if length_name in user_feature_columns:
  user_features[length_name] = Input((1,), name=length_name, dtype='int32')

In [274]:
user_inputs_list = list(user_features.values())
user_inputs_list

[<tf.Tensor 'user_id_2:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'gender_2:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'age_2:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'occupation_2:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'zip_2:0' shape=(None, 1) dtype=int32>,
 <tf.Tensor 'hist_movie_id_2:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_genres_2:0' shape=(None, 50) dtype=int32>,
 <tf.Tensor 'hist_len_2:0' shape=(None, 1) dtype=int32>]

In [275]:
# item feature inputs
item_features = OrderedDict()
for name in item_feature_columns:
  item_features[name] = Input(
                shape=(1,), 
                name=name, 
                dtype="int32"
                )
item_inputs_list = list(item_features.values())


In [276]:
item_inputs_list

[<tf.Tensor 'movie_id_2:0' shape=(None, 1) dtype=int32>]

## Embed Inputs

In [277]:
# Embed sparse inputs
from collections import defaultdict
user_sparse_embeded_input = defaultdict(list)

for name in user_sparse_feature_columns:
  user_sparse_embeded_input[name] = sparse_embedding[name](user_features[name])

In [278]:
# Embed the sequence sparse inputs, need to add pooling layer to it
seq_embeded_dict = defaultdict(list)

for name in user_seq_sparse_feature_columns:
  seq_embeded_dict[name] = sparse_embedding[name](user_features[name])

# from deepctr.layers.sequence import SequencePoolingLayer
# for name in user_seq_sparse_feature_columns:
#   user_seq_sparse_embeded_input[name] = SequencePoolingLayer('mean', supports_masking=False)([seq_embeded_dict[name], user_features[length_name]])

from tensorflow.keras.layers import AveragePooling1D
user_seq_sparse_embeded_input = defaultdict(list)
for name in user_seq_sparse_feature_columns:
  user_seq_sparse_embeded_input[name] = AveragePooling1D(pool_size=50, padding='valid')(seq_embeded_dict[name])

In [279]:
# vec = AveragePooling1D(pool_size=50, strides = 50, padding='valid')
# vec(seq_embeded_dict[name])

In [280]:
# from deepctr.layers.sequence import SequencePoolingLayer
# vec2 = SequencePoolingLayer('mean', supports_masking=False)
# vec2([seq_embeded_dict[name], 35])

In [281]:
# combine embeded inputs into a list
user_sparse_embedding_list = list(user_sparse_embeded_input.values()) + list(user_seq_sparse_embeded_input.values())
user_sparse_embedding_list

[<tf.Tensor 'sparse_emb_user_id_2/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_gender_2/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_age_2/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_occupation_2/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'sparse_emb_zip_2/embedding_lookup/Identity_1:0' shape=(None, 1, 16) dtype=float32>,
 <tf.Tensor 'average_pooling1d_4/Squeeze:0' shape=(None, 1, 32) dtype=float32>,
 <tf.Tensor 'average_pooling1d_5/Squeeze:0' shape=(None, 1, 32) dtype=float32>]

## Combine Embeded Input (User only)

In [282]:
from tensorflow.python.keras.layers import Flatten, Concatenate, Layer, Add

In [283]:
user_dnn_input = Flatten()(Concatenate(axis=2)(user_sparse_embedding_list))
user_dnn_input

<tf.Tensor 'flatten_2/Reshape:0' shape=(None, 144) dtype=float32>

# Sepecify Model Layers

### User Layers

In [284]:
from tensorflow.keras import layers

In [285]:
user_dnn_hidden_units = (128,64, embedding_dim)
dnn_activation='relu'
dnn_use_bn=False
l2_reg_dnn=0
l2_reg_embedding=1e-6
dnn_dropout=0
output_activation='linear'
temperature=0.05,
# sampler_config=sampler_config
seed=1024

In [286]:
from deepctr.layers import DNN

In [287]:
dnn_layers = DNN(hidden_units = user_dnn_hidden_units, 
                  activation='relu', 
                  l2_reg=0, 
                  dropout_rate=0, 
                  use_bn=False, 
                  output_activation='linear',
                  seed=1024
                  )



In [288]:
user_dnn_output = dnn_layers(user_dnn_input)
user_dnn_output = l2_normalize(user_dnn_output)
user_dnn_output

<tf.Tensor 'lambda_6/l2_normalize:0' shape=(None, 32) dtype=float32>

### Item Layer

In [289]:
# Create item index tensor
item_vocabulary_size = feature_max_idx[item_feature_name]
item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(item_features[item_feature_name])

# embed the item index input
item_embeding = sparse_embedding[item_feature_name]
item_embedding_weight = NoMask()(item_embeding(item_index))
pooling_item_embedding_weight = PoolingLayer()([item_embedding_weight])
pooling_item_embedding_weight = l2_normalize(pooling_item_embedding_weight)
pooling_item_embedding_weight


<tf.Tensor 'lambda_7/l2_normalize:0' shape=(3707, 32) dtype=float32>

In [290]:
pooling_item_embedding_weight.eval(session=K.get_session()).shape

(3707, 32)

In [291]:
# .eval(session=K.get_session())

### Output Layer

In [292]:
output = SampledSoftmaxLayer()(
    [pooling_item_embedding_weight, user_dnn_output, item_features[item_feature_name]])
output

<tf.Tensor 'sampled_softmax_layer_2/ExpandDims:0' shape=(None, 1) dtype=float32>

In [293]:
# from model_utils import SampledSoftmaxLayer
# sf = SampledSoftmaxLayer()

In [294]:
# from tensorflow.keras.layers import Softmax
# sf2 = Softmax()
# import numpy as np
# x = np.asarray([[1., 2., 1.]])
# x
# sf2(x).eval(session=K.get_session())

In [295]:
pooling_item_embedding_weight

<tf.Tensor 'lambda_7/l2_normalize:0' shape=(3707, 32) dtype=float32>

In [296]:
user_dnn_output

<tf.Tensor 'lambda_6/l2_normalize:0' shape=(None, 32) dtype=float32>

In [297]:
item_features[item_feature_name]

<tf.Tensor 'movie_id_2:0' shape=(None, 1) dtype=int32>

## Build Model

In [298]:
from tensorflow.python.keras.models import Model

In [299]:
model = Model(inputs=user_inputs_list + item_inputs_list, outputs=output)

model.__setattr__("user_input", user_inputs_list)
model.__setattr__("user_embedding", user_dnn_output)

model.__setattr__("item_input", item_inputs_list)
model.__setattr__("item_embedding",
                  get_item_embedding(pooling_item_embedding_weight, item_features[item_feature_name]))


# Model Compile and Training

In [300]:
model.compile(optimizer="adam", loss=sampledsoftmaxloss)

In [301]:
history = model.fit(train_model_input, train_label,
                    batch_size=512, epochs=20, verbose=1, validation_split=0.0, )

Train on 1976258 samples
Epoch 1/20
1976258/1976258 [==============================] - 124s 63us/sample - loss: 4.8030
Epoch 2/20
1976258/1976258 [==============================] - 122s 62us/sample - loss: 4.7359
Epoch 3/20
1976258/1976258 [==============================] - 122s 62us/sample - loss: 4.7248
Epoch 4/20
1976258/1976258 [==============================] - 124s 63us/sample - loss: 4.7153
Epoch 5/20
1976258/1976258 [==============================] - 122s 62us/sample - loss: 4.7111
Epoch 6/20
1976258/1976258 [==============================] - 121s 61us/sample - loss: 4.7051
Epoch 7/20
1976258/1976258 [==============================] - 122s 62us/sample - loss: 4.7041
Epoch 8/20
1976258/1976258 [==============================] - 121s 61us/sample - loss: 4.7008
Epoch 9/20
1976258/1976258 [==============================] - 121s 61us/sample - loss: 4.7006
Epoch 10/20
1976258/1976258 [==============================] - 120s 61us/sample - loss: 4.6969
Epoch 11/20
1976258/1976258 [=====

In [316]:
res = model.predict(train_model_input)

# Predict: Generate Item and User Embeddings

In [318]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"movie_id": item_profile['movie_id'].values,}

user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2458: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


(6040, 32)
(3706, 32)
time: 489 ms (started: 2022-10-05 21:49:12 +00:00)


In [319]:
user_embs.shape, item_embs.shape

((6040, 32), (3706, 32))

time: 4.39 ms (started: 2022-10-05 21:49:15 +00:00)


In [320]:
prob_matrix = user_embs.dot(item_embs.T)

time: 60.1 ms (started: 2022-10-05 21:49:19 +00:00)


In [321]:
user_idx = 0
a = set([x + 1 for x in prob_matrix[user_idx].argsort()[-50:]])

time: 2.92 ms (started: 2022-10-05 21:49:22 +00:00)


In [322]:
df_train = pd.DataFrame(train_set)

time: 13.3 s (started: 2022-10-05 21:49:23 +00:00)


In [323]:
b = set(df_train[df_train[0] == (user_idx+1)][1].values)

time: 15.1 ms (started: 2022-10-05 21:49:37 +00:00)


In [324]:
len(a&b)

1

time: 11.3 ms (started: 2022-10-05 21:49:37 +00:00)
